In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2

In [ ]:
height = 32
width = 64
size = height*width
path = "C:\VScode_Projects\DP\datasets\CIC-DDoS-2019\clean"
csv_file = path + "\cicddos2019_img.csv"

In [ ]:
df = pd.read_csv(path+"\labeled_sample.csv")

In [ ]:
df = df.drop_duplicates()

In [ ]:
df.info()

In [ ]:
df.columns

In [ ]:
df.protocol.unique()

In [ ]:
df.label.unique()

In [ ]:
len(df.label.unique())

In [ ]:
df.label.value_counts() / len(df)

In [ ]:
downsampled = df[df['label'] == 'UDP'].sample(frac=0.1, random_state=42) 
other_labels = df[df['label'] != 'UDP']
df = pd.concat([downsampled, other_labels])

In [ ]:
df.label.value_counts() / len(df)

In [ ]:
df = df.to_numpy()
df.shape

In [ ]:
# normalize payload, ttl, total_len
for col_index in range(df.shape[1]-3):
    column = df[:, col_index]
    column_normalized = (column - np.min(column)) / (np.max(column) - np.min(column))
    df[:, col_index] = column_normalized

In [ ]:
# inspect t_delta
print(np.min(df[:, -2]), np.max(df[:, -2]))

In [ ]:
# normalize t_delta
column = df[:, -2]
column_normalized = (column - np.min(column)) / (np.max(column) - np.min(column))
df[:, -2] = column_normalized

In [ ]:
def convert_dataset_to_grayscale_image(arr):
    for idx, row in enumerate(arr):
        data = row[:-1]
        label = row[-1]
        print(data.shape)
        print(label)
        
        data = np.pad(data, pad_width=int((size-len(data))/2), constant_values=0)
        print(data.shape)
        data = data.reshape(height, width).astype('float64')
        print(data.shape)
        
        plt.imshow(data, cmap='gray')
        plt.axis('off')  # Remove axes
        plt.show()
        
        break
    print("DONE")
        
convert_dataset_to_grayscale_image(df)

In [ ]:
def convert_dataset_to_rgb_image(arr):
    for idx, row in enumerate(arr):
        data = row[:-1]
        label = row[-1]
        print(data.shape)
        print(label)
        
        data = np.pad(data, pad_width=int((size-len(data))/2), constant_values=0)
        data = data.reshape(height, width)
        
        channel_1 = data.astype('float64')
        print(channel_1.shape)
        channel_2 = np.rot90(channel_1, k=2).reshape(height, width)
        print(channel_2.shape)
        channel_3 = np.rot90(channel_2, k=2).reshape(height, width)
        print(channel_3.shape)
        img = np.stack((channel_1, channel_2, channel_3)).transpose((1, 2, 0))
        print(img.shape)
        
        plt.imshow(img)
        plt.axis('off')  # Remove axes
        plt.show()
        
        break
    print("DONE")
        
convert_dataset_to_rgb_image(df)

In [ ]:
with open(csv_file, 'w') as f:
    np.savetxt(f, [np.array(["file_name", "label"])], delimiter=',', fmt='%s')

def convert_dataset_to_image(arr):
    with open(csv_file, 'a') as f:
        for idx, row in enumerate(arr):
            data = row[:-1]
            label = row[-1]
            if idx % 10_000 == 0:
                    print(idx, label)
            
            data = np.pad(data, pad_width=int((size-len(data))/2), constant_values=0)
            data = data.reshape(height, width)
            
            channel_1 = data.astype('float64')
            channel_2 = np.rot90(channel_1, k=2).reshape(height, width)
            channel_3 = np.rot90(channel_2, k=2).reshape(height, width)
            img = np.stack((channel_1, channel_2, channel_3)).transpose((1, 2, 0))
            
            file_name = f"cic_ddos_2019_{idx}.png"
            cv2.imwrite(path+"\image\\"+file_name, img*255)
            
            log = np.array([file_name, label])
            np.savetxt(f, [log], delimiter=',', fmt='%s')
    print("DONE")
        
convert_dataset_to_image(df)